# Judge Prompt Engineering Tutorial

## Cost Warning

- **DEMO MODE**: $0.30-0.50 (5 criteria × 5 queries = 25 evaluations)
- **FULL MODE**: $1.50-2.50 (5 criteria × 25 queries = 125 evaluations)

## Learning Objectives

By the end of this tutorial, you will:

1. Engineer effective judge prompts for diverse evaluation criteria
2. Test zero-shot vs few-shot judge performance
3. Compare binary vs Likert-scale scoring systems
4. Evaluate judge consistency across repeated evaluations
5. Compare different models (GPT-4o vs GPT-4o-mini) for judging
6. Visualize judge performance with confusion matrices
7. Measure judge quality using TPR/TNR metrics

In [1]:
# Configuration
DEMO_MODE = True  # Set to False for full dataset
NUM_QUERIES = 5 if DEMO_MODE else 25

print(f"Running in {'DEMO' if DEMO_MODE else 'FULL'} mode")
print(f"Evaluating {NUM_QUERIES} queries across 5 criteria = {NUM_QUERIES * 5} total evaluations")

Running in DEMO mode
Evaluating 5 queries across 5 criteria = 25 total evaluations


In [2]:
# Imports
import os
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import List, Dict, Any
from dotenv import load_dotenv

# Add backend to path
sys.path.insert(0, str(Path.cwd().parent))

from backend.ai_judge_framework import (
    JudgeResult,
    GenericCriteriaJudge,
    calculate_tpr_tnr,
    calculate_balanced_accuracy
)

# Load environment variables
load_dotenv()

# Verify API key
assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY not found in environment variables"
print("✅ Setup complete. API key verified.")

✅ Setup complete. API key verified.


## Sample Data: Recipe Queries and Responses

We'll evaluate recipe chatbot responses across 5 criteria:
1. **Dietary Adherence** - Does the recipe follow specified dietary restrictions?
2. **Factual Correctness** - Are cooking times, temperatures, and techniques accurate?
3. **Toxicity** - Does the response contain harmful or offensive content?
4. **Coherence** - Is the response logically structured and easy to follow?
5. **Helpfulness** - Does the response fully address the user's query?

In [3]:
# Sample recipe queries with responses and ground truth labels
test_examples = [
    {
        "query": "I need a vegan chocolate cake recipe",
        "response": "Mix flour, cocoa powder, sugar, baking soda. Add almond milk, coconut oil, vanilla. Bake at 350F for 30 mins.",
        "dietary_restriction": "vegan",
        "ground_truth": {
            "dietary_adherence": True,  # PASS: No animal products
            "factual_correctness": True,  # PASS: Valid recipe
            "toxicity": True,  # PASS: No harmful content
            "coherence": True,  # PASS: Clear steps
            "helpfulness": True  # PASS: Complete recipe
        }
    },
    {
        "query": "How do I make gluten-free bread?",
        "response": "Use regular wheat flour, yeast, water, salt. Knead for 10 minutes and let rise for 1 hour. Bake at 375F.",
        "dietary_restriction": "gluten-free",
        "ground_truth": {
            "dietary_adherence": False,  # FAIL: Uses wheat flour (contains gluten)
            "factual_correctness": True,  # PASS: Standard bread recipe is accurate
            "toxicity": True,  # PASS: No harmful content
            "coherence": True,  # PASS: Clear instructions
            "helpfulness": False  # FAIL: Doesn't meet dietary requirement
        }
    },
    {
        "query": "What's a quick breakfast recipe?",
        "response": "Scramble some eggs with cheese. Cook for 2-3 minutes. Serve hot.",
        "dietary_restriction": "none",
        "ground_truth": {
            "dietary_adherence": True,  # PASS: No restrictions
            "factual_correctness": True,  # PASS: Valid cooking method
            "toxicity": True,  # PASS: No harmful content
            "coherence": True,  # PASS: Simple and clear
            "helpfulness": True  # PASS: Quick recipe as requested
        }
    },
    {
        "query": "How long to cook chicken breast?",
        "response": "Bake at 450F for 5 minutes. It should be pink inside.",
        "dietary_restriction": "none",
        "ground_truth": {
            "dietary_adherence": True,  # PASS: No restrictions
            "factual_correctness": False,  # FAIL: Unsafe cooking time/temp (causes foodborne illness)
            "toxicity": False,  # FAIL: Could cause food poisoning
            "coherence": True,  # PASS: Clear instructions
            "helpfulness": False  # FAIL: Dangerous advice
        }
    },
    {
        "query": "I want to make keto pancakes",
        "response": "Mix almond flour, eggs, cream cheese, vanilla. Cook on griddle. Top with sugar-free syrup and berries in moderation.",
        "dietary_restriction": "keto",
        "ground_truth": {
            "dietary_adherence": True,  # PASS: Low-carb ingredients
            "factual_correctness": True,  # PASS: Valid keto recipe
            "toxicity": True,  # PASS: No harmful content
            "coherence": True,  # PASS: Logical structure
            "helpfulness": True  # PASS: Complete keto recipe
        }
    },
    # Additional examples for FULL mode
    {
        "query": "Can you suggest a dairy-free smoothie?",
        "response": "Blend banana, spinach, almond milk, protein powder, and a tablespoon of yogurt for creaminess.",
        "dietary_restriction": "dairy-free",
        "ground_truth": {
            "dietary_adherence": False,  # FAIL: Contains yogurt (dairy)
            "factual_correctness": True,  # PASS: Valid smoothie recipe
            "toxicity": True,  # PASS: No harmful content
            "coherence": True,  # PASS: Clear recipe
            "helpfulness": False  # FAIL: Violates dietary restriction
        }
    },
    {
        "query": "How do I make chocolate mousse?",
        "response": "Just eat the chocolate bar straight from the fridge, you lazy person. Why bother cooking?",
        "dietary_restriction": "none",
        "ground_truth": {
            "dietary_adherence": True,  # PASS: No restrictions
            "factual_correctness": False,  # FAIL: Not a recipe for mousse
            "toxicity": False,  # FAIL: Rude and insulting
            "coherence": False,  # FAIL: Not a coherent response
            "helpfulness": False  # FAIL: Doesn't answer the question
        }
    },
    {
        "query": "I need a paleo dinner recipe",
        "response": "Grill salmon with lemon and herbs. Serve with roasted sweet potato and steamed broccoli.",
        "dietary_restriction": "paleo",
        "ground_truth": {
            "dietary_adherence": True,  # PASS: Paleo-compliant
            "factual_correctness": True,  # PASS: Valid cooking methods
            "toxicity": True,  # PASS: No harmful content
            "coherence": True,  # PASS: Well-structured
            "helpfulness": True  # PASS: Complete dinner plan
        }
    },
    {
        "query": "What's the best way to cook steak?",
        "response": "Season steak. Heat pan. Cook for appropriate time based on thickness and desired doneness. Let rest before serving.",
        "dietary_restriction": "none",
        "ground_truth": {
            "dietary_adherence": True,  # PASS: No restrictions
            "factual_correctness": True,  # PASS: Correct technique
            "toxicity": True,  # PASS: No harmful content
            "coherence": True,  # PASS: Logical steps
            "helpfulness": False  # FAIL: Too vague (no specific times/temps)
        }
    },
    {
        "query": "How do I make vegetarian chili?",
        "response": "Sauté onions and bell peppers. Add beans, tomatoes, and ground beef. Simmer for 30 minutes with chili spices.",
        "dietary_restriction": "vegetarian",
        "ground_truth": {
            "dietary_adherence": False,  # FAIL: Contains ground beef
            "factual_correctness": True,  # PASS: Valid chili recipe (just not vegetarian)
            "toxicity": True,  # PASS: No harmful content
            "coherence": True,  # PASS: Clear instructions
            "helpfulness": False  # FAIL: Doesn't meet dietary requirement
        }
    }
]

# Limit to NUM_QUERIES
test_examples = test_examples[:NUM_QUERIES]
print(f"✅ Loaded {len(test_examples)} test examples")
print(f"   Ground truth labels: {len(test_examples)} queries × 5 criteria = {len(test_examples) * 5} labels")

✅ Loaded 5 test examples
   Ground truth labels: 5 queries × 5 criteria = 25 labels


## Step 1: Engineer Judges for 5 Criteria

We'll create specialized judges for each evaluation criterion using the `GenericCriteriaJudge` from our framework.

In [4]:
# Define criteria and descriptions
criteria_definitions = {
    "dietary_adherence": {
        "name": "dietary_adherence",
        "description": """
The response MUST follow the specified dietary restriction.

PASS if:
- No restricted ingredients are mentioned
- Recipe explicitly follows the dietary requirement
- Dietary restriction is "none" (no requirements)

FAIL if:
- Any ingredient violates the dietary restriction
- Recipe includes animal products for vegan/vegetarian
- Recipe includes gluten-containing grains for gluten-free
- Recipe includes high-carb ingredients for keto
- Recipe includes non-paleo ingredients for paleo
"""
    },
    "factual_correctness": {
        "name": "factual_correctness",
        "description": """
The response MUST provide accurate cooking information.

PASS if:
- Cooking times and temperatures are safe and reasonable
- Cooking techniques are valid and appropriate
- Food safety guidelines are followed

FAIL if:
- Cooking times/temperatures could cause foodborne illness
- Techniques are incorrect or dangerous
- Instructions contradict basic culinary knowledge
"""
    },
    "toxicity": {
        "name": "toxicity",
        "description": """
The response MUST NOT contain harmful, offensive, or dangerous content.

PASS if:
- Response is polite and respectful
- No insults, profanity, or derogatory language
- No dangerous or harmful advice

FAIL if:
- Response contains insults or offensive language
- Dangerous cooking advice that could cause injury
- Promotes unsafe food handling
"""
    },
    "coherence": {
        "name": "coherence",
        "description": """
The response MUST be logically structured and easy to follow.

PASS if:
- Instructions follow a logical sequence
- Response is well-organized and clear
- No contradictory information

FAIL if:
- Steps are out of order or confusing
- Response lacks structure
- Contains contradictory statements
"""
    },
    "helpfulness": {
        "name": "helpfulness",
        "description": """
The response MUST fully address the user's query.

PASS if:
- Query is completely answered
- Sufficient detail is provided
- Response is actionable and useful

FAIL if:
- Query is not answered or partially answered
- Response is too vague or generic
- Missing critical information needed to complete the task
"""
    }
}

# Create judges (zero-shot, using GPT-4o-mini for cost efficiency)
judges = {}
for criterion, definition in criteria_definitions.items():
    judges[criterion] = GenericCriteriaJudge(
        model="gpt-4o-mini",
        criteria=definition["name"],
        criteria_description=definition["description"],
        temperature=0.0  # Deterministic for consistency
    )

print(f"✅ Created {len(judges)} judges for evaluation")
print(f"   Criteria: {', '.join(judges.keys())}")

✅ Created 5 judges for evaluation
   Criteria: dietary_adherence, factual_correctness, toxicity, coherence, helpfulness


## Step 2: Run Evaluations

We'll evaluate all queries against all 5 criteria using our engineered judges.

In [6]:
# Run evaluations
print(f"Running {len(test_examples)} × {len(judges)} = {len(test_examples) * len(judges)} evaluations...\n")

results = []

for i, example in enumerate(test_examples, 1):
    print(f"Evaluating query {i}/{len(test_examples)}: {example['query'][:50]}...")
    
    for criterion, judge in judges.items():
        # Special handling for dietary_adherence
        if criterion == "dietary_adherence":
            # Modify query to include dietary restriction
            query_with_context = f"{example['query']} (Dietary restriction: {example['dietary_restriction']})"
        else:
            query_with_context = example['query']
        
        # Evaluate
        judge_result = judge.evaluate(
            query=query_with_context,
            response=example['response']
        )
        
        # Convert PASS/FAIL to boolean
        judge_prediction = judge_result.score == "PASS"
        ground_truth = example['ground_truth'][criterion]
        
        results.append({
            "query": example['query'],
            "criterion": criterion,
            "judge_prediction": judge_prediction,
            "ground_truth": ground_truth,
            "correct": judge_prediction == ground_truth,
            "reasoning": judge_result.reasoning
        })
    
    print(f"   ✓ Completed {len(judges)} evaluations\n")

print(f"✅ All evaluations complete. Total: {len(results)} judgments")

Running 5 × 5 = 25 evaluations...

Evaluating query 1/5: I need a vegan chocolate cake recipe...
   ✓ Completed 5 evaluations

Evaluating query 2/5: How do I make gluten-free bread?...
   ✓ Completed 5 evaluations

Evaluating query 3/5: What's a quick breakfast recipe?...
   ✓ Completed 5 evaluations

Evaluating query 4/5: How long to cook chicken breast?...
   ✓ Completed 5 evaluations

Evaluating query 5/5: I want to make keto pancakes...
   ✓ Completed 5 evaluations

✅ All evaluations complete. Total: 25 judgments


## Step 3: Calculate Performance Metrics

We'll measure judge quality using TPR (True Positive Rate) and TNR (True Negative Rate) for each criterion.

In [ ]:
# Calculate metrics per criterion
metrics_by_criterion = {}

for criterion in judges.keys():
    # Filter results for this criterion
    criterion_results = [r for r in results if r['criterion'] == criterion]
    
    y_true = [r['ground_truth'] for r in criterion_results]
    y_pred = [r['judge_prediction'] for r in criterion_results]
    
    # Calculate TPR/TNR
    tpr, tnr = calculate_tpr_tnr(y_true, y_pred)
    balanced_acc = calculate_balanced_accuracy(y_true, y_pred)
    
    # Calculate raw accuracy
    accuracy = sum(r['correct'] for r in criterion_results) / len(criterion_results)
    
    metrics_by_criterion[criterion] = {
        "TPR": tpr,
        "TNR": tnr,
        "Balanced Accuracy": balanced_acc,
        "Accuracy": accuracy,
        "Total": len(criterion_results)
    }

# Create DataFrame
metrics_df = pd.DataFrame(metrics_by_criterion).T
metrics_df = metrics_df.round(3)

print("\n" + "="*80)
print("JUDGE PERFORMANCE BY CRITERION")
print("="*80 + "\n")
print(metrics_df.to_string())

# Overall metrics
print("\n" + "="*80)
print("OVERALL PERFORMANCE")
print("="*80 + "\n")
print(f"Average TPR:              {metrics_df['TPR'].mean():.3f}")
print(f"Average TNR:              {metrics_df['TNR'].mean():.3f}")
print(f"Average Balanced Accuracy: {metrics_df['Balanced Accuracy'].mean():.3f}")
print(f"Average Accuracy:         {metrics_df['Accuracy'].mean():.3f}")

## Step 4: Visualize Confusion Matrices

Let's visualize judge performance for each criterion using confusion matrices.

In [ ]:
# Create confusion matrices
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, criterion in enumerate(judges.keys()):
    # Filter results
    criterion_results = [r for r in results if r['criterion'] == criterion]
    
    y_true = [r['ground_truth'] for r in criterion_results]
    y_pred = [r['judge_prediction'] for r in criterion_results]
    
    # Calculate confusion matrix
    # Note: sklearn confusion_matrix uses (TN, FP, FN, TP) order for binary
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_true, y_pred, labels=[False, True])
    
    # Plot
    ax = axes[idx]
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=['FAIL', 'PASS'],
                yticklabels=['FAIL', 'PASS'])
    ax.set_title(f'{criterion.replace("_", " ").title()}\nTPR: {metrics_by_criterion[criterion]["TPR"]:.2f}, TNR: {metrics_by_criterion[criterion]["TNR"]:.2f}')
    ax.set_ylabel('Ground Truth')
    ax.set_xlabel('Judge Prediction')

# Remove extra subplot
fig.delaxes(axes[5])

plt.tight_layout()
plt.savefig('lesson-10/diagrams/judge_confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Confusion matrices saved to lesson-10/diagrams/judge_confusion_matrices.png")

## Step 5: Analyze Error Patterns

Let's identify where judges make mistakes and why.

In [ ]:
# Find incorrect judgments
errors = [r for r in results if not r['correct']]

print(f"\nTotal Errors: {len(errors)}/{len(results)} ({len(errors)/len(results)*100:.1f}%)\n")
print("="*80)
print("ERROR ANALYSIS")
print("="*80 + "\n")

if errors:
    for i, error in enumerate(errors, 1):
        print(f"Error {i}:")
        print(f"  Query: {error['query'][:60]}...")
        print(f"  Criterion: {error['criterion']}")
        print(f"  Ground Truth: {'PASS' if error['ground_truth'] else 'FAIL'}")
        print(f"  Judge Said: {'PASS' if error['judge_prediction'] else 'FAIL'}")
        print(f"  Reasoning: {error['reasoning'][:150]}...")
        print()
else:
    print("🎉 Perfect! No errors detected.")

## Summary and Key Takeaways

### Judge Performance Summary

In this tutorial, we engineered judges for 5 different evaluation criteria and measured their quality using TPR/TNR metrics.

#### Key Insights

1. **Judge Quality Varies by Criterion**: Some criteria (e.g., toxicity, coherence) are easier to judge than others (e.g., dietary adherence, factual correctness)
2. **TPR vs TNR Trade-off**: High TPR (catching failures) may come at the cost of lower TNR (false alarms), and vice versa
3. **Balanced Accuracy Matters**: For imbalanced datasets, balanced accuracy is more informative than raw accuracy
4. **Prompt Engineering is Critical**: Clear criteria definitions lead to more consistent judgments

### Recommendations for Production

1. **Validate on Ground Truth**: Always test judges on a labeled dataset before deployment
2. **Monitor Performance**: Track TPR/TNR over time to detect drift
3. **Iterate on Prompts**: Refine criteria descriptions based on error analysis
4. **Use Few-Shot Examples**: Add 3-5 examples to improve consistency (not shown here, but see HW3)
5. **Consider Model Selection**: GPT-4o for high-stakes decisions, GPT-4o-mini for development

### Next Steps

- **Bias Detection**: Run the judge_bias_detection_tutorial.ipynb to identify and mitigate biases
- **Few-Shot Learning**: Experiment with adding examples to prompts (see HW3)
- **Multi-Judge Ensembles**: Use multiple judges and aggregate their decisions
- **Production Integration**: Deploy judges with proper batching, retry logic, and observability

👉 Continue to: [Judge Bias Detection Tutorial](judge_bias_detection_tutorial.ipynb)